In [11]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
import pathlib
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [12]:
train_df = pd.read_csv("../input/comp90086-2021/COMP90086_2021_Project_train/train.csv")
train_df

In [13]:
# convert to tuples to pass labels to tf datasets
targets = train_df.iloc[:, 1:3].apply(tuple, axis = 1)
targets

In [14]:
batch_size = 32
img_height = 680
img_width = 490

data_dir = pathlib.Path("../input/comp90086-2021/COMP90086_2021_Project_train")
data_dir

In [15]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  labels=targets.tolist(),
  validation_split=0.2,
  subset="training",
  seed = 123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [16]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  labels=targets.tolist(),
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [17]:
for images, labels in train_ds.take(1):  # only take first element of dataset
    numpy_images = images.numpy()
    numpy_labels = labels.numpy()

In [18]:
numpy_labels

In [19]:
num_classes = 2

model = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

In [20]:
from tensorflow.keras import optimizers
model.compile(loss='mean_squared_error', optimizer=optimizers.RMSprop(lr=1e-4))

In [21]:
history = model.fit(train_ds, epochs=20, batch_size = batch_size, validation_data = val_ds)

In [22]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'])
plt.show()

In [23]:
test_dir = pathlib.Path("../input/comp90086-2021/COMP90086_2021_Project_test")

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  test_dir,
  label_mode=None,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [24]:
preds = model.predict(test_ds)

In [25]:
preds

In [26]:
import csv

test_filenames = []
with open("../input/comp90086-2021/COMP90086_2021_Project_test/imagenames.csv", newline='') as inputfile:
    reader = csv.reader(inputfile)
    # skip header
    header = next(reader)
    
    for row in reader:
        test_filenames.append(str(row[0]))
    
test_filenames[0]

In [27]:
with open("submission.csv","w",newline="") as csvfile:
    writer=csv.writer(csvfile)
    writer.writerow(["id","x","y"])
    index = 0
    for prediction in preds:
        writer.writerow([test_filenames[index], preds[index][0], preds[index][1]])
        index+=1